In [47]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    StaleElementReferenceException,
    TimeoutException,
    WebDriverException,
)
import csv, time, os, requests, glob
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from pathlib import Path
from io import StringIO
from tqdm import tqdm
import psycopg2

DRIVER_PATH = "/Users/kim-youngho/git/GeoNewsApt/notebook/chromedriver-mac-arm64/chromedriver"
yesterday = datetime.today() - timedelta(days=1)
yesterday_str = yesterday.strftime("%Y-%m-%d")
sido = '11000' # 서울 특별시
# CSV 다운을 누르면 연결되는 url
url = 'https://rt.molit.go.kr/pt/xls/ptXlsCSVDown.do;jsessionid=OvMuHixqWmS3QGgERLARxrR-Mwu7T7Fl8UcTJH5f.RT_DN10'
columns_to_use = [
    '시군구', '단지명', '전용면적(㎡)', '계약년월', '계약일',
    '거래금액(만원)', '동', '층', '건축년도', '도로명'
]
folder_path = "/Users/kim-youngho/git/GeoNewsApt/data/raw/apt_sale/"



options = Options()
# GUI 없이 실행 - 백엔드/서버 자동화
options.add_argument("--headless")
# GPU 가속 기능 off - 안정성 개선
options.add_argument("--disable-gpu")
# Chrome을 sandbox 없이 실행함 - 권한 오류 회피(주의)
options.add_argument("--no-sandbox")
options.add_experimental_option("prefs", {
  "download.default_directory": r"/Users/kim-youngho/git/GeoNewsApt/data/raw/apt_sale/",
  "download.prompt_for_download": False,
  "download.directory_upgrade": True,
  "safebrowsing.enabled": True
})

In [48]:
try:
    service = Service(executable_path=DRIVER_PATH)
    driver = webdriver.Chrome(service=service, options=options)
    wait = WebDriverWait(driver, 5)
except WebDriverException as e:
    print(f"[FATAL] Failed to initialize WebDriver: {e}")

try:
    driver.get("https://rt.molit.go.kr/pt/xls/xls.do?mobileAt=")
except Exception as e:
    print(f"[FATAL] Failed to load initial page: {e}")
    driver.quit()

In [51]:
start_day_input = driver.find_element(By.CSS_SELECTOR, '#srhFromDt')
end_day_input = driver.find_element(By.CSS_SELECTOR, '#srhToDt')
sido_select = Select(driver.find_element(By.ID, "srhSidoCd"))
csv_download_btn = driver.find_element(By.CSS_SELECTOR, '#frm_xls > div.quarter-cover > div.quarter-search-if > div.ifdata-btn-box > button:nth-child(2)')

start_day_input.send_keys('2023-04-01')
end_day_input.send_keys('2023-04-01')
sido_select.select_by_value(sido)  # 서울특별시

csv_download_btn.click()

### CSV 열기

In [52]:
# 해당 경로의 모든 .csv 파일 리스트 얻기
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))
df = pd.read_csv(csv_files[0], encoding='cp949', skiprows=15, usecols=columns_to_use)

In [53]:
# 면적당 단가 계산
df['거래금액(만원)'] = df['거래금액(만원)'].str.replace(',', '').astype(int)
df['면적당 단가(만원)'] = df['거래금액(만원)'] / df['전용면적(㎡)']

# 아파트 나이 계산
df['계약년도'] = df['계약년월'].astype(str).str[:4].astype(int)
df['아파트 나이'] = df['계약년도'] - df['건축년도']

#거래 순으로 나열 및 필요 없는 컬럼 삭제
# df['구'] = df['시군구'].str.extract(r'(\S+구)')
# 계약연-월-일을 기준으로 시계열 정렬
df['계약일자'] = df['계약년월'].astype(str) + df['계약일'].astype(str).str.zfill(2)
df['계약일자'] = pd.to_datetime(df['계약일자'], format='%Y%m%d')

df = df.sort_values('계약일자').reset_index(drop=True)
df.drop(['시군구','계약년월','계약일','동','계약년도'], axis=1, inplace=True)

In [54]:
# 이상치 제거 함수 예시 (IQR 방식 등 사용자 정의 필요)
def remove_price_outliers(group):
    q1 = group['거래금액(만원)'].quantile(0.25)
    q3 = group['거래금액(만원)'].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    filtered = group[(group['거래금액(만원)'] >= lower) & (group['거래금액(만원)'] <= upper)]
    return filtered

def calculate_alpha_from_age_count(age, count, N=30):
    """
    아파트 나이(age)와 해당 월 중복 거래 수(count)를 바탕으로
    대표 거래가 산정을 위한 시계열 가중치 α를 계산한다.

    α = min(1, max(0, (1 - age / N) * log2(count + 1)))

    ▣ α 의미:
      - 월평균 거래가(평균값)와 최신 거래가의 가중 평균에서
        평균값에 부여되는 신뢰도 가중치
      - 0 ≤ α ≤ 1 사이 값

    ▣ 설계 목적:
      - 연식이 오래된 아파트일수록 가격 변동성이 크므로,
        최신 거래가(P_latest)에 더 높은 비중을 부여
      - 거래 수가 많을수록 평균값의 신뢰도는 높아지므로 α 증가

    ▣ 기준 수명 N의 역할:
      - 아파트가 노후되기 시작하는 시점을 수치화 (기본값 30년)
      - 한국 도시정비법상 재건축 가능 기준도 30년 → 현실적 기준
        · age = 0 → α 최대 (평균가 신뢰도 최대)
        · age = N → α = 0 (평균가 신뢰도 제거, 최신값만 사용)

    Parameters:
        age (float): 아파트 나이 (년 단위)
        count (int): 해당 월 중복 거래 수
        N (int): 기준 수명 (default: 30년)

    Returns:
        float: 가중치 α (0 ~ 1 범위)
    """
    raw_alpha = (1 - age / N) * np.log2(count + 1)
    alpha = max(0, min(1, raw_alpha))
    return alpha

def representative_price(prices, dates, age, N=30):
    """
    월별 이상치 제거된 거래 가격 리스트와 거래일 리스트,
    아파트 나이를 바탕으로 대표 거래가격을 계산한다.

    대표 거래가 = α * 평균 거래가 + (1 - α) * 최신 거래가

    Parameters:
        prices (list or np.ndarray): 이상치 제거 후의 거래 가격 리스트
        dates (list or np.ndarray): 거래일 리스트 (prices와 길이 동일)
        age (float): 아파트 나이
        N (int): 아파트 기준 수명 (기본 30년)

    Returns:
        float or None: 대표 거래 가격. 거래가 없을 경우 None 반환.
    """
    if len(prices) == 0:
        return None  # 거래 없음 → 대표값 계산 불가

    # 가중치 alpha 계산
    count = len(prices)
    alpha = calculate_alpha_from_age_count(age, count, N)

    # 평균 거래가 (P̄)
    avg_price = np.mean(prices)

    # 최신 거래가 (P_latest) → 가장 나중의 날짜 기준
    latest_index = np.argmax(dates)  # 거래일 기준 최대값 인덱스
    latest_price = prices[latest_index]

    # 대표 거래가 계산
    rep_price = alpha * avg_price + (1 - alpha) * latest_price
    return rep_price


def calculate_alpha_row(group, N=30):
    """
    pandas group (같은 월 내 중복 거래 묶음)을 받아서
    alpha 값을 구하고, 해당 그룹의 첫 row에 붙여 반환.

    Parameters:
        group (pd.DataFrame): 월별 중복 거래 묶음
        N (int): 기준 수명

    Returns:
        pd.DataFrame: alpha가 추가된 대표 row 1개
    """
    age = group['아파트 나이'].iloc[0]  # 해당 그룹의 아파트 나이
    count = len(group)  # 그룹 내 거래 수

    alpha = calculate_alpha_from_age_count(age, count, N)

    # 대표 row는 그룹의 첫 row 기준으로 생성
    row = group.iloc[0].copy()
    row['alpha'] = alpha
    return pd.DataFrame([row])

In [55]:
# 1. 이상치 제거
df_filtered = df.groupby(['도로명', '단지명', '전용면적(㎡)'], group_keys=False)\
                .apply(remove_price_outliers)\
                .reset_index(drop=True)

# 2. 가중치 α 계산 및 대표 row 추출
df = df.groupby(['도로명', '단지명', '전용면적(㎡)'], group_keys=False)\
                  .apply(calculate_alpha_row)\
                  .reset_index(drop=True)

/var/folders/sq/2pgmkw912zj1zpn9tz6pjdmr0000gn/T/ipykernel_86104/1773437094.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(remove_price_outliers)\
/var/folders/sq/2pgmkw912zj1zpn9tz6pjdmr0000gn/T/ipykernel_86104/1773437094.py:8: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(calculate_alpha_row)\


In [56]:
df.drop('거래금액(만원)', axis=1, inplace=True)
df = df.sort_values('계약일자').reset_index(drop=True)


In [57]:
df

,단지명,전용면적(㎡),층,건축년도,도로명,면적당 단가(만원),아파트 나이,계약일자,alpha
0,포시티,14.855,15,2014,면목로 487,686.637496,11,2025-08-22,0.633333
1,"엘지,쌍용아파트",68.120,4,1996,봉화산로 130,866.118614,29,2025-08-22,0.033333
2,하계1청구,84.600,4,1997,한글비석로 91,1004.728132,28,2025-08-22,0.066667
3,중계무지개,59.260,4,1991,동일로208길 19,978.737766,34,2025-08-22,0.000000
4,응암동금호,59.730,14,1998,응암로 318,962.665327,27,2025-08-22,0.100000
5,광장,117.360,11,1978,여의나루로 7,2982.276755,47,2025-08-22,0.000000
6,아스하임,12.880,11,2021,올림픽로 663,1319.875776,4,2025-08-22,0.866667


## 좌표 변환

In [73]:
OUTPUT_PATH = Path('../data/interim/apt/apt_with_long_lat.csv')  
REST_API_KEY = '531d049fba15b8acbb290989f6988d89'

# === 좌표 변환 === #
headers = {'Authorization': f'KakaoAK {REST_API_KEY}'}

def get_coords(address):
    res = requests.get(
        "https://dapi.kakao.com/v2/local/search/address.json",
        headers=headers,
        params={'query': address}
    )
    if res.status_code == 200 and res.json()['documents']:
        doc = res.json()['documents'][0]
        return doc['x'], doc['y']
    return None, None

longitudes, latitudes = [], []
for address in tqdm(df['도로명'], desc="좌표 변환 중"):
    x, y = get_coords(address)
    longitudes.append(x)
    latitudes.append(y)

df['경도'] = longitudes
df['위도'] = latitudes

좌표 변환 중: 100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 20.12it/s]


In [74]:
df

,단지명,전용면적(㎡),층,건축년도,도로명,면적당 단가(만원),아파트 나이,계약일자,alpha,경도,위도
0,포시티,14.855,15,2014,면목로 487,686.637496,11,2025-08-22,0.633333,127.085453248744,37.5953838194036
1,"엘지,쌍용아파트",68.120,4,1996,봉화산로 130,866.118614,29,2025-08-22,0.033333,127.088468055853,37.6031903941314
2,하계1청구,84.600,4,1997,한글비석로 91,1004.728132,28,2025-08-22,0.066667,127.071167444837,37.6398260592996
3,중계무지개,59.260,4,1991,동일로208길 19,978.737766,34,2025-08-22,0.000000,127.065676622278,37.6455956322137
4,응암동금호,59.730,14,1998,응암로 318,962.665327,27,2025-08-22,0.100000,126.921971643797,37.5994661259867
5,광장,117.360,11,1978,여의나루로 7,2982.276755,47,2025-08-22,0.000000,126.920680503797,37.520332613756
6,아스하임,12.880,11,2021,올림픽로 663,1319.875776,4,2025-08-22,0.866667,127.123754054763,37.5408176833646


In [ ]:
# === 최종 정제 및 저장 === #

df['면적당 단가(만원)'] = np.log(df['면적당 단가(만원)'])

# === 수집 못한 위도 경도는 삭제 === #
df.dropna(inplace=True)

# 디렉토리 없으면 생성
OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(OUTPUT_PATH, index=False)

print(f"✅ 저장 완료: {OUTPUT_PATH}")

In [ ]:
df = pd.read_csv('../data/interim/apt/apt_with_long_lat.csv')

### 디비 접속
일단 먼저 구해놨던 정보들 다 디비에 저장하기로 함

In [ ]:

conn = psycopg2.connect(
    host="127.0.0.1",
    port=5432,
    user="postgres",
    password="2464",
    dbname="GeoNewsApt"
)
cur = conn.cursor()

In [ ]:
df.columns

In [ ]:
col_name = ['complex_name','area_m2','floor','built_year',
           'street_name','price_per_m2','apartment_age','contract_day',
            'alpha', 'longitude','latitude']

df.columns=[col_name]

insert_sql = """
INSERT INTO apt (
    complex_name, area_m2, floor, built_year,
    street_name, price_per_m2, apartment_age,
    contract_day, alpha, longitude, latitude
)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""


try:
    for _, row in df.iterrows():
        cur.execute(insert_sql, (
            row['complex_name'],
            row['area_m2'],
            row['floor'],
            row['built_year'],
            row['street_name'],
            row['price_per_m2'],
            row['apartment_age'],
            row['contract_day'],
            row['alpha'],
            row['longitude'],
            row['latitude']
        ))
    conn.commit()

except Exception as e:
    conn.rollback()  # 트랜잭션 초기화
    print("에러:", e)

finally:
    cur.close()
    conn.close()